# Guía de Instalación y Práctica — Sesión 1 (Notebook)
Curso: **IA en el Aula — Nivel Avanzado**  
Profesor: **Luis Daniel Benavides Navarro**  
Fecha: **22 de octubre de 2025**

Este cuaderno guía a los participantes para configurar el entorno, conectarse a una API de modelos de lenguaje y realizar las primeras consultas con **parámetros clave** como `temperature`, `max_tokens`, `top_p`, etc. Se incluyen explicaciones paso a paso y ejercicios.


## 1) Requisitos previos
- Python 3.10 o superior
- Cuenta y **clave API** en un proveedor de modelos de lenguaje (ej. OpenAI)
- Editor/entorno: VSCode, Jupyter o Colab
- Conexión a Internet

Si estás en **Colab**, puedes ejecutar el siguiente comando para instalar dependencias. En local, usa tu terminal.

In [2]:
# (Opcional en local/Colab) Instalar dependencias
%pip install --quiet openai python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\usuario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## 2) Preparar variables de entorno
Crea un archivo `.env` en la carpeta del proyecto con:

```
OPENAI_API_KEY=tu_clave_aqui
```
Nunca publiques tu clave. Evita subir `.env` a repositorios públicos.


In [1]:
# 3) Cargar la clave y crear el cliente
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # Lee el archivo .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("Cliente inicializado. Modelo listo para consultas.")

Cliente inicializado. Modelo listo para consultas.


Si está en Google Colab, debe incluir la llave en los secretos (en el menú de la izquierda), activar la variable y luego inicializar así:

In [2]:
# 3) Cargar la clave y crear el cliente en google Colab
from openai import OpenAI
from google.colab import userdata


client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
print("Cliente inicializado. Modelo listo para consultas.")

ModuleNotFoundError: No module named 'google'

## 4) Parámetros importantes de la API (explicación breve)
- **`model`**: identifica el modelo a consultar (p.ej., `gpt-4o-mini`).
- **`messages`**: lista de turnos conversacionales. Usa roles `system` (instrucciones generales), `user` (tu prompt), `assistant` (respuestas previas si las hay).
- **`temperature`**: *aleatoriedad* de la salida. Valores bajos (0.0–0.3) hacen respuestas más deterministas; valores altos (0.7–1.0) hacen respuestas más creativas.
- **`top_p`**: alternativa a `temperature` basada en muestreo por probabilidad acumulada; usa uno u otro (no ambos a la vez a valores lejanos) para afinar el estilo.
- **`max_tokens`**: tope de tokens generados en la respuesta. Si es muy bajo, el texto puede cortarse.
- **`stop`**: secuencias de corte; si aparecen, la generación se detiene.
- **`frequency_penalty` / `presence_penalty`**: penalizaciones para reducir repeticiones y fomentar aparición de nuevos términos.

**Buenas prácticas:**
- Controlar `temperature` (0.2–0.7) según la tarea (evaluación → bajo; lluvia de ideas → medio/alto).
- Estructurar prompts y, cuando sea útil, **pedir JSON** para facilitar la automatización.
- Evitar incluir datos personales o sensibles en prompts.


In [3]:
# 5) Primera consulta: respuesta libre
prompt = "Explica en dos frases qué es la inteligencia artificial en la educación."
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7  # más creativo que 0.2, menos que 1.0
)
print(response.choices[0].message.content)

La inteligencia artificial en la educación se refiere al uso de tecnologías avanzadas para personalizar y mejorar el proceso de enseñanza y aprendizaje, adaptando recursos y metodologías a las necesidades individuales de los estudiantes. Esto incluye herramientas como tutores virtuales, análisis de datos para evaluar el rendimiento académico y sistemas que facilitan la administración educativa.


### Comentarios sobre `temperature`
- A `0.0–0.2`: respuestas casi siempre iguales (útil en **rubricas** o **explicaciones estándar**).
- A `0.5–0.8`: más variación léxica/estilística (útil en **generación de materiales** o **brainstorming**).
- A `>0.9`: creatividad alta pero riesgo de salidas menos precisas.


In [7]:
# 6) Respuesta estructurada en JSON para automatización
import json

query = "¿Qué es aprendizaje supervisado?"
schema_instruction = (
    "Responde en formato JSON con las claves: operation, input, output. "
    "operation debe ser 'explanation'; input debe repetir la pregunta; output la explicación clara y breve."
)
response_json = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": schema_instruction},
        {"role": "user", "content": query}
    ],
    temperature=0.1,       # más determinista para formatos estructurados
    max_tokens=300         # suficiente para una explicación breve
)
text = response_json.choices[0].message.content
print(text)

# (Opcional) intentar cargar como JSON si el modelo devolvió un objeto válido
try:
    data = json.loads(text)
    print("\nValid JSON →", data)
except json.JSONDecodeError:
    print("\nLa salida no es JSON válido literal. Puedes parsearla manualmente o usar validadores/funciones JSON del proveedor.")

{
  "operation": "explanation",
  "input": "¿Qué es aprendizaje supervisado?",
  "output": "El aprendizaje supervisado es un tipo de aprendizaje automático donde un modelo se entrena utilizando un conjunto de datos etiquetados. Esto significa que cada entrada del conjunto de datos tiene una salida conocida, lo que permite al modelo aprender a predecir la salida a partir de nuevas entradas. Se utiliza comúnmente en tareas como clasificación y regresión."
}

Valid JSON → {'operation': 'explanation', 'input': '¿Qué es aprendizaje supervisado?', 'output': 'El aprendizaje supervisado es un tipo de aprendizaje automático donde un modelo se entrena utilizando un conjunto de datos etiquetados. Esto significa que cada entrada del conjunto de datos tiene una salida conocida, lo que permite al modelo aprender a predecir la salida a partir de nuevas entradas. Se utiliza comúnmente en tareas como clasificación y regresión.'}


## 7) Ejercicios propuestos
1. Cambia `temperature` a 0.1, 0.5 y 0.9 y compara el estilo de las respuestas.
2. Pide que el modelo responda **siempre en JSON** usando un `system` que lo exija. Verifica si cumple.
3. Crea un prompt de tu área (p.ej., *programación*, *arquitectura*, *matemáticas*) que devuelva un JSON con `operation`, `input`, `steps` (lista) y `output`.
4. Limita la longitud con `max_tokens` y observa si corta.


In [10]:
# Plantilla reutilizable para el curso
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.1,
              max_tokens: int = 400,
              system: str | None = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content

# Ejemplo de uso
print(ask_model(
    "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
    temperature=0.4,
    system="Responde en dos oraciones, tono docente y preciso. Siempre responde JSON con llaves promt y respuesta"
))

{
  "prompt": "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
  "respuesta": "Un árbol de decisión es un modelo de aprendizaje automático que utiliza una estructura jerárquica para tomar decisiones basadas en preguntas sobre las características de los datos. Cada nodo interno representa una prueba sobre un atributo, cada rama representa el resultado de la prueba y cada hoja representa una clase o resultado final, permitiendo clasificar o predecir resultados de manera intuitiva."
}


In [11]:
import openai
import os # Importar para obtener la clave API, si se usa un entorno

# ⚠️ ADVERTENCIA: Debes inicializar 'client' correctamente.
# Este es un ejemplo de cómo podrías inicializarlo (ajusta según tu configuración):
# Si usas variables de entorno:
# client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# 💡 Para propósitos de este ejemplo, si no tienes la clave configurada 
# en el entorno, deberás inicializar 'client' o comentar esta línea si 
# la ejecución real ocurre en un entorno donde 'client' ya está definido.
# Si estás ejecutando en un Jupyter/Colab con la clave ya definida, no es necesario.
# Si la variable 'client' no está definida, el código fallará.

# --- Plantilla reutilizable para el curso ---
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.3,
              max_tokens: int = 400,
              system: str | None = None):
    """
    Función para interactuar con un modelo de lenguaje.
    """
    
    # ⚠️ Esta función requiere que la variable 'client' esté definida globalmente
    # y que la conexión a la API sea exitosa.
    try:
        messages = []
        if system:
            messages.append({"role": "system", "content": system})
            
        messages.append({"role": "user", "content": prompt})
        
        resp = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )
        return resp.choices[0].message.content
    except NameError:
        return "ERROR: La variable 'client' (OpenAI client) no está definida. Por favor, inicialízala."
    except Exception as e:
        return f"Error al comunicarse con la API: {e}"

# --- Definiciones para el Ejemplo de Uso (Física) ---

# 🎯 PROMPT DE FÍSICA: Cálculo de Energía Cinética.
prompt_fisica = "Calcula la energía cinética de un coche que tiene una masa de 1500 kg y se mueve a una velocidad constante de 20 m/s. Formula la ley física utilizada."

# 🤖 SYSTEM: Exige el formato JSON estricto.
system_json_request = """
Tu única tarea es resolver el problema de Física.
DEBES responder únicamente con un objeto JSON que contenga las siguientes claves:
1. 'operation': La ley física utilizada o el objetivo del cálculo (e.g., 'Cálculo de Energía Cinética').
2. 'input': Los datos iniciales proporcionados (e.g., 'm=1500 kg, v=20 m/s').
3. 'steps': Una lista de cadenas, donde cada elemento detalla una etapa del cálculo o la formulación de la ley. Incluye la fórmula.
4. 'output': El resultado final del cálculo con unidades (e.g., '300000 J').
NO incluyas texto adicional fuera del objeto JSON.
"""

# --- Ejemplo de Uso y Ejecución ---

print("--- Ejecutando Modelo para Cálculo de Energía Cinética ---")
print(ask_model(
    prompt=prompt_fisica,
    temperature=0.1, # Baja temperatura para consistencia y precisión
    system=system_json_request
))
print("----------------------------------------------------------")

--- Ejecutando Modelo para Cálculo de Energía Cinética ---
{
  "operation": "Cálculo de Energía Cinética",
  "input": "m=1500 kg, v=20 m/s",
  "steps": [
    "La fórmula de la energía cinética es: EK = (1/2) * m * v^2",
    "Sustituyendo los valores: EK = (1/2) * 1500 kg * (20 m/s)^2",
    "Calculando: EK = (1/2) * 1500 kg * 400 m^2/s^2",
    "EK = 750 kg * 400 m^2/s^2",
    "EK = 300000 kg*m^2/s^2",
    "EK = 300000 J"
  ],
  "output": "300000 J"
}
----------------------------------------------------------


In [12]:
import openai
import os
import json # Importar para una mejor manipulación de JSON

# ⚠️ ADVERTENCIA: Debes inicializar 'client' correctamente.
# Si usas la librería de OpenAI, la variable 'client' debe estar definida 
# con tu clave API para que el código se ejecute correctamente.
# Ejemplo (ajusta según tu entorno):
# client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# --- Plantilla reutilizable para el curso ---
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.3,
              max_tokens: int = 400,
              system: str | None = None):
    """
    Función para interactuar con un modelo de lenguaje.
    """
    
    # ⚠️ Esta función requiere que la variable 'client' esté definida.
    try:
        messages = []
        if system:
            messages.append({"role": "system", "content": system})
            
        messages.append({"role": "user", "content": prompt})
        
        # Simulación de la respuesta para entornos sin la clave API configurada
        # Si 'client' está definido y la clave es válida, se ejecutará la llamada real.
        if 'client' not in globals() and 'client' not in locals():
            print("AVISO: 'client' no está definido. Devolviendo una respuesta de ejemplo simulada.")
            return json.dumps({
                "operation": "Cálculo de Elasticidad Precio de la Demanda (EPD)",
                "input": "Precio inicial ($10), Cantidad inicial (100 unidades), Precio final ($12), Cantidad final (90 unidades)",
                "steps": [
                    "Fórmula de la EPD (Método del Punto Medio): EPD = [(Qf - Qi) / ((Qf + Qi)/2)] / [(Pf - Pi) / ((Pf + Pi)/2)].",
                    "Cálculo del cambio porcentual en la cantidad demandada: [(90 - 100) / ((90 + 100)/2)] = -10 / 95 ≈ -0.1053.",
                    "Cálculo del cambio porcentual en el precio: [(12 - 10) / ((12 + 10)/2)] = 2 / 11 ≈ 0.1818.",
                    "Cálculo de la EPD: -0.1053 / 0.1818 ≈ -0.579."
                ],
                "output": "La Elasticidad Precio de la Demanda es aproximadamente -0.58. La demanda es inelástica porque el valor absoluto (|0.58|) es menor que 1."
            }, indent=2)

        # Llamada real a la API (si 'client' está definido)
        resp = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )
        return resp.choices[0].message.content
        
    except NameError:
        return "ERROR: La variable 'client' (OpenAI client) no está definida. Por favor, inicialízala."
    except Exception as e:
        return f"Error al comunicarse con la API: {e}"

# --- Definiciones para el Ejemplo de Uso (Economía) ---

# 🎯 PROMPT DE ECONOMÍA: Cálculo de la EPD.
prompt_economia = "Calcula la Elasticidad Precio de la Demanda utilizando el método del punto medio si el precio de un bien sube de $10 a $12, causando que la cantidad demandada baje de 100 unidades a 90 unidades. Indica qué tipo de elasticidad es."

# 🤖 SYSTEM: Exige el formato JSON estricto.
system_json_request = """
Tu única tarea es resolver el problema de Economía y proporcionar la interpretación del resultado.
DEBES responder únicamente con un objeto JSON que contenga las siguientes claves:
1. 'operation': La ley económica utilizada (e.g., 'Cálculo de Elasticidad Precio de la Demanda (EPD)').
2. 'input': Los datos iniciales proporcionados (e.g., 'Pi=$10, Qi=100, Pf=$12, Qf=90').
3. 'steps': Una lista de cadenas, donde cada elemento detalla una etapa del cálculo, la fórmula y la interpretación.
4. 'output': El resultado numérico final y el tipo de elasticidad.
NO incluyas texto adicional, explicaciones ni encabezados fuera del objeto JSON.
"""

# --- Ejemplo de Uso y Ejecución ---

print("--- Ejecutando Modelo para Cálculo de Elasticidad Precio de la Demanda ---")
print(ask_model(
    prompt=prompt_economia,
    temperature=0.1, # Muy baja temperatura para asegurar precisión numérica y formato
    system=system_json_request
))
print("--------------------------------------------------------------------------")

--- Ejecutando Modelo para Cálculo de Elasticidad Precio de la Demanda ---
{
  "operation": "Cálculo de Elasticidad Precio de la Demanda (EPD) utilizando el método del punto medio",
  "input": "Pi=$10, Qi=100, Pf=$12, Qf=90",
  "steps": [
    "1. Calcular el cambio en la cantidad demandada: ΔQ = Qf - Qi = 90 - 100 = -10.",
    "2. Calcular el cambio en el precio: ΔP = Pf - Pi = 12 - 10 = 2.",
    "3. Calcular el promedio de la cantidad demandada: Qprom = (Qi + Qf) / 2 = (100 + 90) / 2 = 95.",
    "4. Calcular el promedio del precio: Pprom = (Pi + Pf) / 2 = (10 + 12) / 2 = 11.",
    "5. Aplicar la fórmula de la elasticidad: EPD = (ΔQ / Qprom) / (ΔP / Pprom) = (-10 / 95) / (2 / 11).",
    "6. Calcular EPD: EPD = (-0.1053) / (0.1818) = -0.58.",
    "7. Interpretación: La elasticidad precio de la demanda es -0.58, lo que indica que la demanda es inelástica."
  ],
  "output": {
    "value": -0.58,
    "type": "inelástica"
  }
}
---------------------------------------------------------------

In [13]:
import openai
import os
import json

# ⚠️ ADVERTENCIA: Debes inicializar 'client' correctamente.
# Este código asume que la variable 'client' está definida globalmente
# y la conexión a la API está configurada.

# --- Plantilla reutilizable para el curso ---
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.1,
              max_tokens: int = 400, # Valor por defecto más común
              system: str | None = None):
    """
    Función para interactuar con un modelo de lenguaje y observar el efecto de max_tokens.
    """
    
    # Manejo básico de la ausencia de 'client' para simulación
    try:
        if 'client' not in globals() and 'client' not in locals():
            raise NameError("La variable 'client' no está definida.")
            
        messages = []
        if system:
            messages.append({"role": "system", "content": system})
            
        messages.append({"role": "user", "content": prompt})
        
        resp = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens # 👈 El valor que se va a limitar
        )
        return resp.choices[0].message.content
        
    except NameError:
        # Respuesta simulada si 'client' no está definido
        simulated_response = (
            "La fusión nuclear es el proceso mediante el cual dos núcleos atómicos ligeros se combinan para formar un núcleo más pesado. Este proceso libera una enorme cantidad de energía, mucho mayor que la fisión nuclear. Ocurre naturalmente en el Sol y las estrellas, donde las altas temperaturas y presiones superan la repulsión coulómbica de los núcleos. En la Tierra, se busca replicar esta reacción en reactores de confinamiento magnético (como los tokamaks) o inercial, utilizando isótopos de hidrógeno (deuterio y tritio) como combustible, para crear una fuente de energía limpia y prácticamente inagotable. Es el santo grial de la energía."
        )
        if max_tokens < len(simulated_response.split()):
            return f"SIMULACIÓN DE CORTE (max_tokens={max_tokens}): {simulated_response[:max_tokens*5]}..." # Aprox. 5 caracteres por token en español
        return simulated_response
    except Exception as e:
        return f"Error al comunicarse con la API: {e}"

# --- Definiciones para el Ejemplo de Uso ---

# 💬 PROMPT que pide una respuesta LAAAARGA para forzar el corte.
prompt_largo = "Explica detalladamente qué es la fusión nuclear, dónde ocurre, por qué es importante y los principales desafíos para su implementación en la Tierra. Necesito una explicación extensa."

# 🤖 SYSTEM: Instrucción de tono simple.
system_instruccion = "Responde con un tono informativo y académico."


# --- Ejemplo de Uso y Verificación ---

print("--- 1. Ejecución con max_tokens BAJO (40 tokens) ---")
# El modelo debería detenerse abruptamente en medio de una frase.
print(ask_model(
    prompt=prompt_largo,
    temperature=0.1, # Baja para que la respuesta sea coherente hasta el corte
    max_tokens=40,   # 👈 Valor bajo para forzar el corte
    system=system_instruccion
))

print("\n" + "="*50 + "\n")

print("--- 2. Ejecución con max_tokens ALTO (400 tokens) ---")
# El modelo debería completar toda la explicación.
print(ask_model(
    prompt=prompt_largo,
    temperature=0.1, 
    max_tokens=400, # 👈 Valor alto, suficiente para la respuesta
    system=system_instruccion
))
print("\n" + "="*50)

--- 1. Ejecución con max_tokens BAJO (40 tokens) ---
La fusión nuclear es un proceso físico en el cual dos núcleos atómicos ligeros se combinan para formar un núcleo más pesado, liberando una cantidad significativa de energía en el proceso


--- 2. Ejecución con max_tokens ALTO (400 tokens) ---
La fusión nuclear es un proceso físico en el cual dos núcleos atómicos ligeros se combinan para formar un núcleo más pesado, liberando una cantidad significativa de energía en el proceso. Este fenómeno es el responsable de la producción de energía en las estrellas, incluida nuestra propia estrella, el Sol.

### Proceso de Fusión Nuclear

La fusión nuclear ocurre cuando los núcleos de dos átomos ligeros, como el hidrógeno, se acercan lo suficiente como para superar la repulsión electrostática que existe entre ellos debido a sus cargas positivas. Para que esto suceda, es necesario alcanzar condiciones extremas de temperatura y presión, que permiten que los núcleos se fusionen. En el caso del Sol, 

In [14]:
import openai
import json
import os 
# ⚠️ ADVERTENCIA: Debes inicializar 'client' correctamente.
# Si estás en un entorno donde 'client' no está definido, el código usará
# la respuesta simulada para demostrar el corte.

# --- Plantilla reutilizable para el curso ---
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.1,
              max_tokens: int = 400, # Valor por defecto
              system: str | None = None):
    """
    Función para interactuar con un modelo de lenguaje.
    """
    
    # Simulación de la respuesta si 'client' no está definido,
    # para demostrar el efecto del max_tokens.
    if 'client' not in globals() and 'client' not in locals():
        # Simulamos una respuesta que debería ser larga, pero se corta a 40 tokens.
        simulated_response = {
            "prompt": prompt,
            "respuesta_completa_esperada": "La arquitectura Von Neumann es un diseño fundamental que utiliza una única memoria para almacenar tanto las instrucciones del programa como los datos que serán procesados. Esto simplifica la estructura de control, pero puede generar un cuello de botella...",
            "respuesta_simulada_cortada": "La arquitectura Von Neumann utiliza una única memoria para almacenar tanto las instrucciones del programa como los datos que serán procesados. Esto simplifica la estructura de control, pero puede generar un cuello de botella conocido como el 'cuello de botella de Von Neumann', que limita la velocidad de procesamiento al tener que acceder a una misma ruta para ambas tareas. Esta limitación afecta directamente el rendimiento...",
            "respuesta_cortada_a_40_tokens": "La arquitectura Von Neumann utiliza una única memoria para almacenar tanto las instrucciones del programa como los datos que serán procesados. Esto simplifica la estructura de control, pero puede generar un cuello de botella conocido como el 'cuello de botella de Von Neumann', que limita la velocidad de procesamiento al tener que acceder a una misma ruta para"
        }
        
        # Devolvemos la respuesta cortada simulada para el ejercicio
        # Nótese que el modelo real NO genera un JSON incompleto, sino que corta el texto.
        if max_tokens <= 40:
             # Devolvemos el texto cortado para demostrar el efecto
             # Ignoramos temporalmente la exigencia de JSON para mostrar el corte puro.
            return simulated_response["respuesta_cortada_a_40_tokens"] + "..." 
        else:
            return "Simulación de respuesta larga sin corte."
        
    # Si 'client' está definido, se ejecuta la llamada real a la API
    try:
        messages = []
        if system:
            messages.append({"role": "system", "content": system})
            
        messages.append({"role": "user", "content": prompt})
        
        resp = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens # Aquí aplicamos el límite
        )
        return resp.choices[0].message.content
        
    except Exception as e:
        return f"Error: La llamada real falló. {e}"

# --- Definiciones para el Ejemplo de Uso (Informática) ---

# 🎯 PROMPT DE INFORMÁTICA: Pide una explicación detallada.
prompt_informatica = "Explica en detalle los principios fundamentales de la arquitectura Von Neumann y por qué se la conoce como el 'cuello de botella'."

# 🤖 SYSTEM: Instrucción de tono (manteniendo el tono)
system_request = "Responde en un tono técnico y detallado. La respuesta debe ser lo más extensa posible para probar el límite de tokens."

# --- Ejemplo de Uso con max_tokens BAJO (40) ---

LIMITE_DE_TOKENS = 40

print("--- Ejecutando Modelo con Límite de Tokens (max_tokens=40) ---")
print(f"Pregunta: {prompt_informatica}")
print(f"Límite de tokens establecido: {LIMITE_DE_TOKENS}\n")

respuesta_cortada = ask_model(
    prompt=prompt_informatica,
    temperature=0.1,
    max_tokens=LIMITE_DE_TOKENS, # 🚨 Aquí se aplica el límite bajo
    system=system_request
)

print("### RESPUESTA CORTADA ###")
print(respuesta_cortada)
print("\n--- Observación ---")
print("Como puedes ver, la respuesta se interrumpe abruptamente (usualmente a mitad de una frase) porque el modelo alcanzó el límite de 40 tokens antes de terminar la explicación o de cerrar cualquier estructura (como un JSON o una frase completa).")

# --- Ejemplo de Uso con max_tokens ALTO (400) ---
print("\n--- Ejecutando Modelo con Límite ALTO (max_tokens=400) para comparación ---")

respuesta_completa = ask_model(
    prompt=prompt_informatica,
    temperature=0.1,
    max_tokens=400,
    system=system_request
)

print("\n### RESPUESTA COMPLETA (Simulada o Real) ###")
print(respuesta_completa)
print("----------------------------------------------------------")

--- Ejecutando Modelo con Límite de Tokens (max_tokens=40) ---
Pregunta: Explica en detalle los principios fundamentales de la arquitectura Von Neumann y por qué se la conoce como el 'cuello de botella'.
Límite de tokens establecido: 40

### RESPUESTA CORTADA ###
La arquitectura Von Neumann es un modelo de diseño de computadoras que fue propuesto por el matemático y físico John von Neumann en la década de 1940. Este modelo ha sido fundamental

--- Observación ---
Como puedes ver, la respuesta se interrumpe abruptamente (usualmente a mitad de una frase) porque el modelo alcanzó el límite de 40 tokens antes de terminar la explicación o de cerrar cualquier estructura (como un JSON o una frase completa).

--- Ejecutando Modelo con Límite ALTO (max_tokens=400) para comparación ---

### RESPUESTA COMPLETA (Simulada o Real) ###
La arquitectura Von Neumann es un modelo de diseño de computadoras que fue propuesto por el matemático y físico John von Neumann en la década de 1940. Este modelo ha s

## 9) Solución de problemas comunes
- **`openai.AuthenticationError` / `401`**: clave inválida o no cargada; revisa tu `.env` y reinicia el kernel.
- **`Rate limit`**: excediste el número de solicitudes por minuto; espera unos segundos y reintenta.
- **`model_not_found`**: el modelo no existe o no tienes acceso; cambia a uno disponible en tu cuenta.
- **Salidas no JSON**: fija `temperature=0.0–0.3`, agrega instrucciones `system` estrictas y/o usa funciones nativas de validación JSON si el proveedor las ofrece.


## 10) Próximos pasos
En la siguiente sesión se verán **consultas avanzadas**, manejo de contexto y el punto de partida para construir un **asistente con RAG** (recuperación de conocimiento) usando materiales del curso.
